In [15]:
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, AdamW

import json
import pickle as pkl
from pycparser import c_generator
import ForPragmaExtractor.visitors as visitor
from Model import tokenizer

VAR_PREFIX = "var"
ARR_PREFIX = "arr"
FUNC_PREFIX = "func"
STRUCT_PREFIX = "struct"
generator = c_generator.CGenerator()
id_v = visitor.CounterIdVisitor()
replacer = visitor.ReplaceIdsVisitor(VAR_PREFIX, ARR_PREFIX, STRUCT_PREFIX, FUNC_PREFIX)

# -------------------------------------------------------------------------




In [16]:
def db_read_string_from_file(file_path):
    try:
        with open(file_path, "r") as f:
            return "".join(f.readlines())   # readlines()是读全部
    except:
        return ""


def normalize_code_as_ast(pickle_file):
    # print (pickle_file)
    with open(pickle_file, 'rb') as f:
        pragmafor_tuple = pkl.load(f)  #
        for_ast = pragmafor_tuple.for_node
        # for_ast.show()
        # print(normalize_code_as_string.generator.visit(for_ast))
        # for_ast.show()
        # counts in an array the name and identifiers of the code
        id_v.reset()
        id_v.visit(for_ast)
        # Replace the names..
        replacer.reset(id_v.ids, id_v.array,id_v.struct, id_v.func)
        replacer.visit(for_ast)
        with open('temp.txt', 'w') as f:
            for_ast.show(buf=f)
        with open('temp.txt', 'r') as f:
            ast = f.readlines()

        ast_no_whitespaces = [a.strip() for a in ast] # kill all whitespaces
        # print(ast_no_whitespaces)
        # print(normalize_code_as_string.generator.visit(for_ast))

        # print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        ast_one_line = " " + " ".join(ast_no_whitespaces)
        return ast_one_line


# str1 = normalize_code_as_ast('DB_TEST/DB_TEST/database/PolyBench-ACC-master_.gitignore_2mm.c_5/code_pickle.pkl')
# print(str1)

def code_as_ast(pickle_file):
    # print (pickle_file)
    with open(pickle_file, 'rb') as f:
        pragmafor_tuple = pkl.load(f)  #
        for_ast = pragmafor_tuple.for_node
        with open('temp.txt', 'w') as f:
            for_ast.show(buf=f)
        with open('temp.txt', 'r') as f:
            ast = f.readlines()

        ast_no_whitespaces = [a.strip() for a in ast] # kill all whitespaces and \n
        ast_one_line = " " + " ".join(ast_no_whitespaces)
        return ast_one_line


str2 = code_as_ast('D:\CodeLibrary\PragFormer-main\DB_TEST\DB_TEST\database\PolyBench-ACC-master_.gitignore_2mm.c_5\code_pickle.pkl')
# print(str2)
# print('str2len: ', len(str2))

data1 = {'text': [str2]}
print("data1:\n", data1)

text, _ = tokenizer.deepscc_tokenizer(data1['text'])
print("after tokenized:\n", text)
print('token_len:', len(text.input_ids[0]), len(text.attention_mask[0]))

print("vocab_size: ", _)
# -------------------------------------------------------------------------



data1:
 {'text': [" For: Assignment: = ID: i Constant: int, 0 BinaryOp: < ID: i ID: ni UnaryOp: p++ ID: i For: Assignment: = ID: j Constant: int, 0 BinaryOp: < ID: j ID: nl UnaryOp: p++ ID: j Assignment: = ArrayRef: ArrayRef: ID: D ID: i ID: j BinaryOp: / BinaryOp: * Cast: Typename: None, [] TypeDecl: None, [] IdentifierType: ['double'] ID: i BinaryOp: + ID: j Constant: int, 2 ID: nk"]}
after tokenized:
 {'input_ids': [[0, 286, 35, 46091, 35, 5457, 4576, 35, 939, 33685, 35, 6979, 6, 321, 47466, 13926, 35, 28696, 4576, 35, 939, 4576, 35, 10265, 1890, 1766, 13926, 35, 181, 42964, 4576, 35, 939, 286, 35, 46091, 35, 5457, 4576, 35, 1236, 33685, 35, 6979, 6, 321, 47466, 13926, 35, 28696, 4576, 35, 1236, 4576, 35, 295, 462, 1890, 1766, 13926, 35, 181, 42964, 4576, 35, 1236, 46091, 35, 5457, 42719, 31842, 35, 42719, 31842, 35, 4576, 35, 211, 4576, 35, 939, 4576, 35, 1236, 47466, 13926, 35, 1589, 47466, 13926, 35, 1009, 6719, 35, 5957, 9675, 4344, 35, 9291, 6, 48081, 7773, 45788, 35, 9291, 6, 

In [17]:
"""
    数据处理
"""
data_set = {'text':[], 'label':[], 'ast':[]}
jsonpath = 'D:\CodeLibrary\PragFormer-main\DB_TEST\DB_TEST\database.json'
with open(jsonpath, 'r') as f:
    file_data = json.load(f)
    for i, key in enumerate(file_data):
        code = db_read_string_from_file(file_data[key]["code"])
        ast_str = code_as_ast(file_data[key]['code_pickle'])
        if file_data[key]['pragma']:
            pragma = db_read_string_from_file(file_data[key]['pragma'])
        else:
            pragma = '0'

        data_set['text'].append(code)
        data_set['ast'].append(ast_str)
        data_set['label'].append(pragma)

# print("data_set['text'][0]\n", data_set['text'][0])
# print("data_set['text'][10]\n", data_set['text'][10])
# print("data_set['ast'][0]\n", data_set['ast'][0])
# print("data_set['ast'][10]\n", data_set['ast'][10])
# print("data_set['label'][0]\n", data_set['label'][0])
# print("data_set['label'][10]\n", data_set['label'][10])

def label_encoder(labels:list):
    new_labels = []
    for label in labels:
        if 'omp parallel for' in label and 'private' not in label and 'reduction' not in label:
            new_labels.append('1')
        elif 'omp parallel for' in label and 'private' in label and 'reduction' not in label:
            new_labels.append('2')
        elif 'omp parallel for' in label and 'reduction' in label and 'private' not in label:
            new_labels.append('3')
        elif 'omp parallel for' in label and 'reduction' in label and 'private' in label:
            new_labels.append("4")
        else:
            new_labels.append("0")
    return new_labels


# def collate_fn(batch):
#     return {
#         'input_ids': pad_sequence([item['input_ids'] for item in batch], batch_first=True),
#         'attention_mask': pad_sequence([item['attention_mask'] for item in batch], batch_first=True),
#         'labels': torch.tensor([item['labels'] for item in batch])
#     }
#
print(data_set['label'])
data_set['label'] = label_encoder(data_set['label'])
print(data_set['label'])




['omp parallel for private(i, j) schedule(dynamic, blocksize)', 'omp parallel for private(i, j) schedule(dynamic, blocksize)', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', '0', '0', '0', '0', '0', '0', '0', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', '0', '0', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for schedule(dynamic,4) shared(status)', 'omp parallel for sched

In [18]:


# Split the dataset into train and test sets
texts_train, texts_test, asts_train, asts_test, labels_train, labels_test = train_test_split(
    data_set['text'], data_set['ast'], data_set['label'], test_size=0.2, random_state=42
) 


# Define a custom dataset class
class CodeDataset(Dataset):
    def __init__(self, texts, asts, labels):
        self.texts = texts
        self.asts = asts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        ast = self.asts[idx]
        label = int(self.labels[idx])  # Assuming labels are numerical

        # Tokenize input
        inputs = tokenizer(text, ast, return_tensors='pt', padding='max_length', truncation=True, max_length=512)

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }
    

# Create dataset and dataloaders for training and testing
train_dataset = CodeDataset(texts_train, asts_train, labels_train)
test_dataset = CodeDataset(texts_test, asts_test, labels_test)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [20]:


# Define the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
model = model.half()

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

OutOfMemoryError: CUDA out of memory. Tried to allocate 46.00 MiB. GPU 0 has a total capacty of 8.00 GiB of which 0 bytes is free. Of the allocated memory 7.26 GiB is allocated by PyTorch, and 43.62 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0.0  # Track total training loss for the epoch
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

        # Release GPU cache
        torch.cuda.empty_cache()

    # Calculate average training loss for the epoch
    avg_train_loss = total_train_loss / len(train_loader)

    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}')

# Validation loop (optional)
    model.eval()
    total_test_loss = 0.0  # Track total testing loss for the epoch
    predictions = []
    true_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            total_test_loss += loss.item()

            # Get model predictions
            logits = outputs.logits
            _, predicted_labels = torch.max(logits, dim=1)
            predictions.extend(predicted_labels.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calculate average testing loss for the epoch
    avg_test_loss = total_test_loss / len(test_loader)

    # Calculate accuracy on the testing set
    accuracy = accuracy_score(true_labels, predictions)

    print(f'Epoch {epoch + 1}/{num_epochs}, Testing Loss: {avg_test_loss:.4f}, Testing Accuracy: {accuracy:.4f}')